In [1]:
from PIL import Image
from multiprocessing import Pool

from tqdm import tqdm

import cv2
import numpy as np
import pandas as pd
import rarfile

from mtcnn import MTCNN

Using TensorFlow backend.
/home/agni-ubuntu/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/agni-ubuntu/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/agni-ubuntu/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/agni-ubuntu/anaconda3/lib/python3.7

In [41]:
detector = MTCNN() 

def detect_face(img): 
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB) 
    final = [] 
    detected_faces_raw = detector.detect_faces(img)
    
    if detected_faces_raw==[]: 
        print('no faces found') 
        return [] 
    
    for x in detected_faces_raw: 
        x,y,w,h=x['box'] 
        final.append([x,y,w,h]) 
        return final 

In [42]:
def crop(img,x,y,w,h): 
    x-=40 
    y-=40 
    w+=40 
    h+=40 
    if x<0: 
        x=0 
    if y<=0: 
        y=0 
    return cv2.resize(img[y:y+h,x:x+w],(92,92))

In [43]:
def get_frames(file_name):
    cap = cv2.VideoCapture(file_name)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    _step = frame_count // 25
    _next = _step
    count = 0 
    frames = list()
    
    while count < 25:
        ret, frame = cap.read()
        if (ret != True):
            break
        if ret:
            frame_no = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            
            if frame_no == _next:
                bb = detect_face(frame)
                if bb:
                    x,y,w,h = bb[0]
                    img = crop(frame, x,y,w,h)
                    frames.append(np.array(img))
                    _next = frame_no + _step
                    count += 1
    
    cap.release()
    
    return frames


In [44]:
rf = rarfile.RarFile("UCF101.rar")

In [45]:
file_paths = [f.filename for f in rf.infolist() if f.filename.endswith(".avi")]
file_names = [fp.split("/")[-1] for fp in file_paths]

In [10]:
pool = Pool(8)

process = pool.map(get_frames, file_paths)

pool.close()
pool.join()

In [55]:
for v_num, v in tqdm(enumerate(process)):
    for i_num, i in enumerate(v):
        img = Image.fromarray(i)
        img.save("training_data/real/img_{}_{}.jpg".format(v_num,i_num))

13320it [03:11, 69.54it/s]


In [2]:
import glob
import random

In [3]:
real_file_list = glob.glob("training_data/real/*.jpg")
fake_file_list = glob.glob("training_data/fake/*.jpg")

In [4]:
data = list()
labels = list()

In [5]:
for item in fake_file_list:
    img = cv2.imread(item)
    resized = cv2.resize(img, (92,92), interpolation = cv2.INTER_AREA)
    data.append(resized)
    labels.append(0)

In [6]:
real_data_count = len(fake_file_list)

real_sampling = random.choices(real_file_list, k=real_data_count)

for item in real_sampling:
    img = cv2.imread(item)
    resized = cv2.resize(img, (92,92), interpolation = cv2.INTER_AREA)
    data.append(resized)
    labels.append(1)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_x, test_x, train_y, test_y = train_test_split(np.array(data), np.array(labels), test_size=0.20)

In [9]:
from keras.applications.mobilenet import MobileNet
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import Adam

In [10]:
input = Input(shape=(92, 92, 3))
base_model = MobileNet(input_shape=(92, 92, 3), alpha=1, include_top=False, pooling='avg')

for i, layer in enumerate(base_model.layers):
    # When 0, the first layer i.e. the input layer should be trainable=False
    if i > 88:
        layer.trainable = True
    else:
        layer.trainable = False
        
x = base_model(input)
x = Dense(1, activation='sigmoid', name='final_output')(x)

model = Model(input, x)

model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['crossentropy', 'accuracy'])

model.fit(train_x, train_y, epochs=3, batch_size=32)

Instructions for updating:
Colocations handled automatically by placer.


/home/agni-ubuntu/anaconda3/lib/python3.7/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Instructions for updating:
Use tf.cast instead.
Epoch 1/3
73281/73281 [==============================] - 1169s 16ms/step - loss: 0.0390 - crossentropy: 0.0390 - accuracy: 0.9860
Epoch 2/3
73281/73281 [==============================] - 1176s 16ms/step - loss: 0.0169 - crossentropy: 0.0169 - accuracy: 0.9945
Epoch 3/3
73281/73281 [==============================] - 1174s 16ms/step - loss: 0.0136 - crossentropy: 0.0136 - accuracy: 0.9954


In [11]:
model.save("vid_detect_model.h5")

In [12]:
predict = model.predict(np.array(test_x))

In [13]:
new_pred = [round(float(i)) for i in predict]

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
accuracy_score(new_pred, np.array(test_y))*100

60.07314011243928

In [16]:
def video_classifier(frame_predictions):
    new_pred = [round(float(i)) for i in frame_predictions]
    ones = zeros = 0
    
    for pred in new_pred:
        if pred == 1:
            ones += 1
        else:
            zeros += 1
            
    if ones >= zeros:
        return 1
    return 0
    

In [17]:
for v_num, v in tqdm(enumerate(test_x)):
    for i_num, i in enumerate(v):
        img = Image.fromarray(i)
        img.save("output/test_img/img_{}_{}.jpg".format(v_num,i_num))

18321it [04:03, 75.27it/s]


In [18]:
test_file_list = glob.glob("output/test_img/*.jpg")

df = pd.DataFrame(list(zip(test_file_list, new_pred)), columns =['The_Name_of_Img', 'Is_This_Img_real?']) 

In [20]:
df.to_csv("output/output.csv", index=None)